# Apply template-to-template RheMAP tranforms to afids
Applying ANTs-generated transforms to Slicer-generated .fcsv files
## Preparations
### Imports

In [2]:
import os
from itertools import combinations
from registration_utils import transform_fcsv

### Define paths 

In [13]:
# prefixes (RheMAP and AFIDs use different prefixes for the templates)
rhemap_prefixes = ['NMTv1.3', 'NMTv2.0_stereo_asym', 'D99', 'INIA', 'MNI', 'YRK']
afids_prefixes = ['nmtv1.3', 'nmtv2.0asym', 'd99', 'inia19', 'macaqueMNI', 'yerkes19']
# make dictionary matching RheMAP prefixes to AFIDs prefixes
prefix_dict = {key: value for (key, value) in zip(rhemap_prefixes, afids_prefixes)}

# Warps (Replace with your local RheMAP warp paths)
warps = '/home/niko/MRI/RheMAP/warps'
warps_linear = os.path.join(warps, 'linear') # linear: Affine only
warps_final = os.path.join(warps, 'final') # final: composite Affine + SyN

# Get afids-macaca root directory
root = os.path.dirname(os.getcwd())

# Folder with MEAN PHASE1 post-QC .fcsv files
# (1 set of consensus afids per template)
afids_inputs_means = os.path.join(root, 'data', 'PHASE1_output_afid_postQC')

# Folder with ALL PHASE1 post-QC .fcsv files
afids_inputs_all = os.path.join(root, 'data', 'PHASE1_input_afid_postQC')
# List of path to ALL QC .fcsv files
all_fcsv_inputs = [os.path.join(afids_inputs_all, f)
                   for f in os.listdir(afids_inputs_all) 
                   if f.endswith('.fcsv')]
all_fcsv_inputs.sort()

# Folders for storing output PHASE 4 .fcsv files
afids_outputs = os.path.join(root, 'data', 'PHASE4_RheMAP_transformed')
afids_outputs_means = os.path.join(afids_outputs, 'MEAN')
afids_outputs_all = os.path.join(afids_outputs, 'ALL')

## Apply transforms to points
The transforms are run iteratively between all unique template pairs.
* First, the MEAN QC .fcsv files are transformed
* Second, ALL QC .fcsv files (for all raters and sessions) are transformed
  
**Important note**: ANTs transforms on points work in the opposite direction (as compared to image transforms).

For example, let's imagine we have to go from space A to space B either using a linear transform (.mat) or a non-linear warp(.nii.gz)
* Linear A-to-B transform for images: `antsApplyTransforms -i A -r B -o A-in-B -t A-to-B.mat`
* Linear A-to-B transform for points: `antsApplyTransformsToPoints -i A -r B -o A-in-B -t [A-to-B.mat, 1]` (inverting the forward transform)
* Nonlinear A-to-B transform for images: `antsApplyTransforms -i A -r B -o A-in-B -t A-to-B_1Warp.nii.gz` (for RheMAP we would use the `A-to-B_CompositeWarp.nii.gz`)
* Nonlinear A-to-B transform for points: `antsApplyTransformsToPoints -i A -r B -o A-in-B A-to-B_1InverseWarp.nii.gz` (for RheMAP we would use the `B-to-A_CompositeWarp.nii.gz`)

The application of the `antsApplyTransformsToPoints` command to Slicer-generated .fcsv files is implemented in the `transform_fcsv` function (imported from `registration_utils.py`). The function takes the following arguments:
1. input fcsv: path to input Slicer fcsv file
2. ouput fcsv: path to output Slicer fcsv file
3. transform: path to ANTs transform file (either linear .mat or .nii.gz warp)
4. invert: if 1, linear .mat is inverted

In [14]:
# Iterate over unique template pairs
for rheA, rheB in combinations(rhemap_prefixes, 2):

    print(f'Transforming fiducials between {rheA} and {rheB}...')
    # Get appropriate RheMAP warp paths
    warp_linear_AtoB = os.path.join(warps_linear, f'{rheA}_to_{rheB}_affine_0GenericAffine.mat')
    warp_final_AtoB = os.path.join(warps_final, f'{rheA}_to_{rheB}_CompositeWarp.nii.gz')
    warp_final_BtoA = os.path.join(warps_final, f'{rheB}_to_{rheA}_CompositeWarp.nii.gz')

    # Get prefixes in AFIDs spelling
    afdA, afdB = prefix_dict[rheA], prefix_dict[rheB]

    ###################################
    # Transform MEAN QC consensus files
    ###################################
    # Input .fcsv files
    inA_mean = os.path.join(afids_inputs_means, f'{afdA}_MEAN_QC.fcsv')
    inB_mean = os.path.join(afids_inputs_means, f'{afdB}_MEAN_QC.fcsv')
    
    # Run transforms between templates
    # A-to-B linear
    transform_fcsv(inA_mean,
                   os.path.join(afids_outputs_means, f'{afdA}_MEAN_QC_in_{afdB}_linear.fcsv'),
                   warp_linear_AtoB, invert=1)
    # A-to-B final
    transform_fcsv(inA_mean,
                   os.path.join(afids_outputs_means, f'{afdA}_MEAN_QC_in_{afdB}_final.fcsv'),
                   warp_final_BtoA, invert=0)
    # B-to-A linear5 templates x 8 raters x 2 sessions
    transform_fcsv(inB_mean, 
                   os.path.join(afids_outputs_means, f'{afdB}_MEAN_QC_in_{afdA}_linear.fcsv'),
                   warp_linear_AtoB, invert=0)
    # B-to-A final
    transform_fcsv(inB_mean,
                   os.path.join(afids_outputs_means, f'{afdB}_MEAN_QC_in_{afdA}_final.fcsv'),
                   warp_final_AtoB, invert=0)
    print('Transformed MEAN QC .fcsv files!')

    ###################################
    # Transform ALL QC .fcsv files
    ###################################
    # List of all PHASE1 post-QC .fcsv inputs in template A space
    all_inputs_A = [f for f in all_fcsv_inputs if os.path.basename(f).split('_')[1] == afdA]
    # Transform them from template A to template B
    for inA in all_inputs_A:
        outA_prefix = os.path.basename(inA).split('.fc')[0]
        # A-to-B linear
        transform_fcsv(inA,
                       os.path.join(afids_outputs_all, f'{outA_prefix}_in_{afdB}_linear.fcsv'),
                       warp_linear_AtoB, invert=1)
        # A-to-B final
        transform_fcsv(inA,
                       os.path.join(afids_outputs_all, f'{outA_prefix}_in_{afdB}_final.fcsv'),
                       warp_final_BtoA, invert=0)

    # List of all PHASE1 post-QC .fcsv inputs in template B space
    all_inputs_B = [f for f in all_fcsv_inputs if os.path.basename(f).split('_')[1] == afdB]
    # Transform them from template B to template A
    for inB in all_inputs_B:
        outB_prefix = os.path.basename(inB).split('.fc')[0]
        # B-to-A linear
        transform_fcsv(inB,
                       os.path.join(afids_outputs_all, f'{outB_prefix}_in_{afdA}_linear.fcsv'),
                       warp_linear_AtoB, invert=0)
        # B-to-A final
        transform_fcsv(inB,
                       os.path.join(afids_outputs_all, f'{outB_prefix}_in_{afdA}_final.fcsv'),
                       warp_final_AtoB, invert=0)
                       
    print('Transformed ALL QC .fcsv files!')

Transforming fiducials between NMTv1.3 and NMTv2.0_stereo_asym...
Transformed MEAN QC .fcsv files!
Transformed ALL QC .fcsv files!
Transforming fiducials between NMTv1.3 and D99...
Transformed MEAN QC .fcsv files!
Transformed ALL QC .fcsv files!
Transforming fiducials between NMTv1.3 and INIA...
Transformed MEAN QC .fcsv files!
Transformed ALL QC .fcsv files!
Transforming fiducials between NMTv1.3 and MNI...
Transformed MEAN QC .fcsv files!
Transformed ALL QC .fcsv files!
Transforming fiducials between NMTv1.3 and YRK...
Transformed MEAN QC .fcsv files!
Transformed ALL QC .fcsv files!
Transforming fiducials between NMTv2.0_stereo_asym and D99...
Transformed MEAN QC .fcsv files!
Transformed ALL QC .fcsv files!
Transforming fiducials between NMTv2.0_stereo_asym and INIA...
Transformed MEAN QC .fcsv files!
Transformed ALL QC .fcsv files!
Transforming fiducials between NMTv2.0_stereo_asym and MNI...
Transformed MEAN QC .fcsv files!
Transformed ALL QC .fcsv files!
Transforming fiducials bet